In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 140kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.3
    Uninstalling scipy-1.3.3:
      Successfully uninstalled scipy-1.3.3


In [0]:
import os
import numpy as np
from os import listdir
from scipy.misc import imread, imresize
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Settings:
img_size = 64
grayscale_images = False
num_class = 10
test_size = 0.2


def get_img(data_path):
    # Getting image array from path:
    img = imread(data_path, flatten=grayscale_images)
    img = imresize(img, (img_size, img_size, 1 if grayscale_images else 3))
    
    return img

def get_dataset(dataset_path='drive/My Drive/Sign-Language-Digits-Dataset-master (1)/Dataset (1)'):
    # Getting all data from data path:
    try:
        X = np.load('npy_dataset/X.npy')
        Y = np.load('npy_dataset/Y.npy')
    except:
        labels = listdir(dataset_path) # Geting labels
        X = []
        Y = []
        for i, label in enumerate(labels):
            datas_path = dataset_path+'/'+label
            for data in listdir(datas_path):
                img = get_img(datas_path+'/'+data)
                X.append(img)
                Y.append(i)
        # Create dateset:
        X -= np.mean(X)
        X = np.array(X).astype('float32')/255.
        Y = np.array(Y).astype('float32')
        Y = to_categorical(Y, num_class)
        if not os.path.exists('npy_dataset/'):
            os.makedirs('npy_dataset/')
        np.save('npy_dataset/X.npy', X)
        np.save('npy_dataset/Y.npy', Y)
    X, X_test, Y, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    return X, X_test, Y, Y_test

if __name__ == '__main__':
    get_dataset()

In [0]:
 import numpy as np
 import matplotlib.pyplot as plt
 X = np.load('npy_dataset/X.npy') 
 Y = np.load('npy_dataset/Y.npy') 
# f, ax = plt.subplots(2, 2, figsize=(15, 10))
# sample = [290, 1000, 1800, 650]
# for i in range(0, 4):
#     ax[i//2, i%2].imshow(X[sample[i]].reshape(64, 64,3))
#     ax[i//2, i%2].axis('on')
# plt.show()

In [10]:
X.shape
X = X.reshape(X.shape[0], 64, 64, 3)
X.shape
X_train=[]
X_test=[]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=None)
print(X_train.shape[0])

1649


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
def model1(X_train,y_train,X_test,y_test):
  model_input = Input(shape=(64, 64,3))
  feature = Conv2D(32, (3, 3), activation='relu', padding='same')(model_input)
  feature = MaxPooling2D(pool_size=(2, 2))(feature)    
  feature = Dropout(0.5)(feature)
    
  feature = Conv2D(32, (3, 3), activation='relu', padding='same')(feature)
  feature = Conv2D(16, (3, 3), activation='relu', padding='same')(feature)

  feature = MaxPooling2D(pool_size=(2, 2))(feature)    
  feature = Dropout(0.5)(feature)
    
  feature = Conv2D(32, (3, 3), activation='relu', padding='same')(feature)       
  feature = MaxPooling2D(pool_size=(2, 2))(feature)    
  feature = MaxPooling2D(pool_size=(2, 2))(feature)    

  feature = Dropout(0.5)(feature)
    
  feature = Flatten()(feature)
  feature = Dense(512, activation='relu')(feature)    
  feature = Dropout(0.5)(feature)
    
  output = Dense(10, activation='softmax')(feature)
    
  model = Model(inputs=model_input, outputs= output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m, precision_m, recall_m])
  model.summary()
 
  n_folds=4

  for i in range(n_folds):
      print("Training on Fold: ",i+1)
      t_x, val_x, t_y, val_y = train_test_split(X_train, y_train, test_size=0.25, random_state = 7)
      model.fit(t_x, t_y, validation_data=(val_x, val_y), epochs=10, batch_size=32)
      model.evaluate(val_x, val_y)


     
  average=0

  for i in range(len(X_test)):
   ypre=model.predict(X_test[i].reshape(1,64,64,3))
   if(ypre.any()==y_test[i].any()):
     average = average+1
  print("test predictions ",average/len(y_test))
  loss, accuracy, scores, pre, recall = model.evaluate(X_test, y_test)

  return scores, pre, recall,average/len(y_test)

  
def model2(X_train,y_train,X_test,y_test):
  model_input = Input(shape=(64, 64,3))
  features = Conv2D(64, (3, 3), activation='relu', padding='same')(model_input)
  features = MaxPooling2D(pool_size=(2, 2))(features)   
  features = MaxPooling2D(pool_size=(2, 2))(features)    
 
  features = Dropout(0.5)(features)
    
  features = Conv2D(64, (3, 3), activation='relu', padding='same')(features)
  features = Conv2D(64, (3, 3), activation='relu', padding='same')(features)

  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
    
  features = Flatten()(features)
  features = Dense(512, activation='relu')(features)    
  features = Dropout(0.5)(features)
    
  output = Dense(10, activation='softmax')(features)
    
  model = Model(inputs=model_input, outputs= output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m, precision_m, recall_m])
  model.summary()
 
  n_folds=4

  for i in range(n_folds):
      print("Training on Fold: ",i+1)
      t_x, val_x, t_y, val_y = train_test_split(X_train, y_train, test_size=0.25, random_state = 7)
      model.fit(t_x, t_y, validation_data=(val_x, val_y), epochs=10, batch_size=32)
      model.evaluate(val_x, val_y)
  loss, accuracy, scores, pre, recall = model.evaluate(X_test, y_test)
  
     
  average=0

  for i in range(len(X_test)):
   ypre=model.predict(X_test[i].reshape(1,64,64,3))
   if(ypre.any()==y_test[i].any()):
     average = average+1
  print("predictions ",average/len(y_test))
  return scores, pre, recall,average/len(y_test)

def model3(X_train,y_train,X_test,y_test):
  model_input = Input(shape=(64, 64,3))
  features = Conv2D(16, (2, 2), activation='relu', padding='same')(model_input)
  features= MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
    
  features = Conv2D(32, (3, 3), activation='relu', padding='same')(features)
  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
  features = Conv2D(64, (5, 5), activation='relu', padding='same')(features)
  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
  features = Conv2D(128, (5, 5), activation='relu', padding='same')(features)
  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
    
  features = Flatten()(features)
  features = Dense(512, activation='relu')(features)    
  features = Dropout(0.5)(features)
    
  output = Dense(10, activation='softmax')(features)
    
  model = Model(inputs=model_input, outputs= output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m, precision_m, recall_m])
  model.summary()
 
  n_folds=4

  for i in range(n_folds):
      print("Training on Fold: ",i+1)
      t_x, val_x, t_y, val_y = train_test_split(X_train, y_train, test_size=0.25, random_state = 7)
      model.fit(t_x, t_y, validation_data=(val_x, val_y), epochs=10, batch_size=32)
      model.evaluate(val_x, val_y)
  loss, accuracy, scores, pre, recall = model.evaluate(X_test, y_test)
  
  average=0

  for i in range(len(X_test)):
   ypre=model.predict(X_test[i].reshape(1,64,64,3))
   if(ypre.any()==y_test[i].any()):
     average = average+1
  print("predictions ",average/len(y_test))
   
  return scores, pre, recall,average/len(y_test)
def model4(X_train,y_train,X_test,y_test):
  model_input = Input(shape=(64, 64,3))
  features = Conv2D(128, (3, 3), activation='relu', padding='same')(model_input)
  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)
    
  features = Conv2D(128, (3, 3), activation='relu', padding='same')(features)
  features = MaxPooling2D(pool_size=(2, 2))(features)    
  features = Dropout(0.5)(features)    
  features = Flatten()(features)
  features = Dense(64, activation='relu')(features)    
  features = Dropout(0.5)(features)
    
  output = Dense(10, activation='softmax')(features)
    
  model = Model(inputs=model_input, outputs= output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m, precision_m, recall_m])
  model.summary()
 
  n_folds=4

  for i in range(n_folds):
      print("Training on Fold: ",i+1)
      t_x, val_x, t_y, val_y = train_test_split(X_train, y_train, test_size=0.25, random_state = 7)
      history=model.fit(t_x, t_y, validation_data=(val_x, val_y), epochs=10, batch_size=32)
      model.evaluate(val_x, val_y) 
  loss, accuracy, scores, pre, recall = model.evaluate(X_test, y_test)
  
     
  average=0

  for i in range(len(X_test)):
   ypre=model.predict(X_test[i].reshape(1,64,64,3))
   if(ypre.any()==y_test[i].any()):
     average = average+1
  print("predictions ",average/len(y_test))
   
  return scores, pre, recall,average/len(y_test)

In [0]:
print(model1(X_train,y_train,X_test,y_test))
print(model2(X_train,y_train,X_test,y_test))
print(model3(X_train,y_train,X_test,y_test))
print(model4(X_train,y_train,X_test,y_test))



Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_70 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 16)        4624      
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 16, 16, 16)        0  